In [1]:
import pandas as pd

In [ ]:
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
report_mapping = pd.read_csv("/report_mapping.csv")
sample_submission_format = pd.read_csv("/sample_submission_format.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [3]:
pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=6564eb412a0ecdd9f2e2ab385e1462abf327929437dc90e9222986de6327b004
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [43]:
pip install ace_tools

In [5]:
import docx2txt
import pdfplumber

In [61]:
import os
import re
import pandas as pd
import docx2txt
import pdfplumber
from openai import AzureOpenAI
import json

# Azure OpenAI credentials
gpt4o_endpoint = "https://fuchsopenai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2023-03-15-preview"
gpt4o_key = "377635a0dfdd4f01a1352ea785ea4537"

# Azure OpenAI model and client configuration
model_name = "gpt-4o-2024-08-06"
client = AzureOpenAI(
    azure_endpoint=gpt4o_endpoint,
    api_key=gpt4o_key,
    api_version="2024-02-01"
)

# Load the mapping files
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

def extract_report_id(file_path):
    file_name = os.path.basename(file_path)
    try:
        report_id = int(re.search(r'Report_(\d+)', file_name).group(1))
        return report_id
    except (AttributeError, ValueError):
        print(f"Error: Could not extract report ID from file name: {file_name}")
        return None

def extract_text_from_file(file_path):
    file_extension = file_path.split('.')[-1].lower()
    report_id = extract_report_id(file_path)

    try:
        if file_extension == 'txt':
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read(), report_id

        elif file_extension == 'docx':
            return docx2txt.process(file_path), report_id

        elif file_extension == 'pdf':
            extracted_text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    extracted_text += page.extract_text() or ""
            return extracted_text, report_id

        elif file_extension == 'xlsx':
            excel_data = pd.ExcelFile(file_path)
            sheet_names = excel_data.sheet_names
            combined_data = ""

            for sheet in sheet_names:
                if "report" in sheet.lower():  # Skip instruction sheets
                    continue
                else:
                    sheet_data = pd.read_excel(file_path, sheet_name=sheet)
                    combined_data += sheet_data.to_string(index=False) + "\n"

            return combined_data, report_id

        else:
            raise ValueError(f"Unsupported file type: {file_extension}")

    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None, report_id

def openai_sequence_response(prompt_list):
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=prompt_list
        )
        response_text = response.choices[0].message.content
        return response_text

    except Exception as e:
        print(f"Error generating text: {e}")
        return None

def fix_report_id_in_response(api_response, correct_report_id):
    try:
        response_data = json.loads(api_response)
        for record in response_data:
            record['report_id'] = correct_report_id
        return json.dumps(response_data, indent=2)
    except json.JSONDecodeError:
        print("Error: Failed to parse API response as JSON.")
        print("Raw API response:", api_response)  # Print raw response for debugging
        return None

def map_ids(response_data):
    mapped_data = []

    for record in response_data:
        mapped_record = record.copy()

        # Map machine_id
        machine_name = record['machine_id']
        if pd.notna(machine_name):
            machine_name_cleaned = machine_name.strip().lower()
            machine_id_row = machine_mapping.loc[
                machine_mapping['machine_name'].str.lower().str.strip() == machine_name_cleaned
            ]
            if not machine_id_row.empty:
                mapped_record['machine_id'] = machine_id_row['machine_id'].values[0]
            else:
                mapped_record['machine_id'] = None
                print(f"Warning: Could not map machine name '{machine_name}' to machine_id.")
        else:
            mapped_record['machine_id'] = None

        # Map product1_id, product2_id, product3_id
        for i in range(1, 4):
            product_name = record.get(f'product{i}_id')
            if product_name:
                product_name_cleaned = product_name.strip().lower()
                product_id_row = product_mapping.loc[
                    product_mapping['product_name'].str.lower().str.strip() == product_name_cleaned
                ]
                if not product_id_row.empty:
                    mapped_record[f'product{i}_id'] = product_id_row['product_id'].values[0]
                else:
                    mapped_record[f'product{i}_id'] = None
                    print(f"Warning: Could not map product name '{product_name}' to product_id.")
            else:
                mapped_record[f'product{i}_id'] = None

        # Map unit
        unit_name = record['unit']
        if unit_name:
            unit_name_cleaned = unit_name.strip().lower()
            unit_id_row = unit_mapping.loc[
                unit_mapping['unit_name'].str.lower().str.strip() == unit_name_cleaned
            ]
            if not unit_id_row.empty:
                mapped_record['unit'] = unit_id_row['unit_id'].values[0]
            else:
                mapped_record['unit'] = None  # If not found
                print(f"Warning: Could not map unit name '{unit_name}' to unit_id.")
        else:
            mapped_record['unit'] = None

        mapped_data.append(mapped_record)

    return mapped_data

def create_prompt_with_report_id(report_id):
    return f"""You are an experienced Data Science Artificial Intelligence Agent with the ability to find and gather insights\
in the wide array of data types, noisy and dirty. You are provided with a test analysis report from FUCHS. FUCHS is now the world's largest \
independent supplier of innovative lubrication solutions, covering almost every industry and application. The report has \
Different machines (sometimes called instruments) which are capable of doing analysis on a bunch of ingredients (sometimes called products)\
and the conducted experimental value associated with that. You are tasked to provide the report_id, products used in that report, and the machine\
used in the report and summarize the results in a tabulated or JSON output. The table should look like a JSON-formatted output described below.\
Do not provide the steps but rather analyze the data yourself and provide the final output as described.
[
  {{
    "report_id": 1,
    "machine_id": 0,
    "product1_id": 0,
    "product2_id": null,
    "product3_id": null,
    "result": 10,
    "unit": 0
  }}
]
Following is the text from the report_{report_id}
"""

# File path (replace with your actual file path)
file_path = '/Report_57.xlsx'

# Extract text and report_id from the file based on its type
file_text_content, report_id = extract_text_from_file(file_path)

# Check if the extracted text and report_id are valid
if file_text_content is None or report_id is None:
    print("Error: Could not extract the content or report ID from the file.")
else:
    # Create the dynamic prompt with the correct report ID
    example_prompt = create_prompt_with_report_id(report_id)

    # Example of a prompt list with multiple steps
    example_prompt_list = [
        {"role": "system", "content": example_prompt},
        {"role": "user", "content": file_text_content}
    ]

    # Execute the sequence prompt function
    response_sequence = openai_sequence_response(example_prompt_list)

    # Print the raw API response for debugging
    print("Raw API response:", response_sequence)

    # Ensure the correct report_id is used in the API response
    fixed_response = fix_report_id_in_response(response_sequence, report_id)

    if fixed_response is None:
        print("Error: Failed to fix the report ID in the response.")
    else:
        try:
            # Load the API response into a Python object
            response_data = json.loads(fixed_response)
            print("Successfully parsed API response.")

            # Map machine, product, and unit names to IDs
            mapped_data = map_ids(response_data)

            # Generate the DataFrame
            df_mapped = pd.DataFrame(mapped_data)

            # Print the output to the terminal
            print("Mapped Data Output:")
            print(df_mapped.to_string(index=False))

            # Generate the final CSV
            output_csv_file = 'final_submission.csv'
            df_mapped.to_csv(output_csv_file, index=False)
            print(f"Final submission CSV generated: {output_csv_file}")

        except json.JSONDecodeError as e:
              print(f"Error: Failed to parse API response as JSON. Error: {e}")
              print(f"Raw API response: {fixed_response}")



Raw API response: ```json
[
  {
    "report_id": 57,
    "machine_id": "NMR-500",
    "product1_id": "Coconut Oil",
    "product2_id": "Beeswax",
    "product3_id": "Glycerin",
    "result": 18.5,
    "unit": "ppm"
  },
  {
    "report_id": 57,
    "machine_id": "FTIR-8400",
    "product1_id": "Almond Oil",
    "product2_id": "Gum",
    "product3_id": "Vitamin E",
    "result": 3420,
    "unit": "1/cm"
  },
  {
    "report_id": 57,
    "machine_id": "PCR-96",
    "product1_id": "Almond Oil",
    "product2_id": "Cetyl Alcohol",
    "product3_id": null,
    "result": 28,
    "unit": "Ct"
  },
  {
    "report_id": 57,
    "machine_id": "LC-400",
    "product1_id": "Almond Oil",
    "product2_id": null,
    "product3_id": null,
    "result": 125.5,
    "unit": "ug/mL"
  },
  {
    "report_id": 57,
    "machine_id": "UV-2600",
    "product1_id": "Jojoba Oil",
    "product2_id": "Gum",
    "product3_id": "Glycerin",
    "result": 2.8,
    "unit": "Abs"
  },
  {
    "report_id": 57,
    "mach

In [70]:
import os
import re
import pandas as pd
import docx2txt
import pdfplumber
import json
from openai import AzureOpenAI

# Azure OpenAI credentials
gpt4o_endpoint = "https://fuchsopenai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2023-03-15-preview"
gpt4o_key = "377635a0dfdd4f01a1352ea785ea4537"

# Azure OpenAI model and client configuration
model_name = "gpt-4o-2024-08-06"
client = AzureOpenAI(
    azure_endpoint=gpt4o_endpoint,
    api_key=gpt4o_key,
    api_version="2024-02-01"
)

# Load the mapping files
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

# Function to send a request to OpenAI to find the closest match
def get_closest_machine_name_openai(machine_name, available_machines):
    """
    Function to get the closest machine name using OpenAI's language model.

    Parameters:
    - machine_name (str): The machine name from the input data.
    - available_machines (list): List of machine names from the mapping file.

    Returns:
    - str: Closest matching machine name as suggested by OpenAI.
    """
    # Build the prompt
    prompt = f"""
    I have a list of available machine names: {', '.join(available_machines)}.
    Based on this list, find the closest matching machine name for: "{machine_name}".
    Only provide the closest match.
    """

    # Call the OpenAI API
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}]
        )
        response_text = response.choices[0].message.content.strip()
        return response_text
    except Exception as e:
        print(f"Error generating text from OpenAI: {e}")
        return None

def map_ids_using_openai(response_data):
    mapped_data = []

    # Available machine names from the machine mapping file
    available_machine_names = machine_mapping['machine_name'].tolist()

    for record in response_data:
        mapped_record = record.copy()

        # Map machine_id using OpenAI
        machine_name = record['machine_id']
        if pd.notna(machine_name):
            machine_name_cleaned = machine_name.strip().lower()

            # Get the closest machine name using OpenAI
            closest_machine_name = get_closest_machine_name_openai(machine_name_cleaned, available_machine_names)

            if closest_machine_name:
                # Fetch the corresponding machine_id for the closest match
                machine_id_row = machine_mapping.loc[
                    machine_mapping['machine_name'].str.lower().str.strip() == closest_machine_name.lower()
                ]
                if not machine_id_row.empty:
                    mapped_record['machine_id'] = machine_id_row['machine_id'].values[0]
                else:
                    mapped_record['machine_id'] = None
                    print(f"Warning: Could not map machine name '{machine_name}' to machine_id.")
            else:
                mapped_record['machine_id'] = None
                print(f"Warning: Could not find a close match for machine name '{machine_name}' using OpenAI")
        else:
            mapped_record['machine_id'] = None

        # Map product1_id, product2_id, product3_id
        for i in range(1, 4):
            product_name = record.get(f'product{i}_id')
            if product_name:
                product_name_cleaned = product_name.strip().lower()
                product_id_row = product_mapping.loc[
                    product_mapping['product_name'].str.lower().str.strip() == product_name_cleaned
                ]
                if not product_id_row.empty:
                    mapped_record[f'product{i}_id'] = product_id_row['product_id'].values[0]
                else:
                    mapped_record[f'product{i}_id'] = None
                    print(f"Warning: Could not map product name '{product_name}' to product_id.")
            else:
                mapped_record[f'product{i}_id'] = None

        # Map unit
        unit_name = record['unit']
        if unit_name:
            unit_name_cleaned = unit_name.strip().lower()
            unit_id_row = unit_mapping.loc[
                unit_mapping['unit'].str.lower().str.strip() == unit_name_cleaned
            ]
            if not unit_id_row.empty:
                mapped_record['unit'] = unit_id_row['unit_id'].values[0]
            else:
                mapped_record['unit'] = None  # If not found
                print(f"Warning: Could not map unit name '{unit_name}' to unit_id.")
        else:
            mapped_record['unit'] = None

        mapped_data.append(mapped_record)

    return mapped_data

# Dummy JSON Data: Assuming you get this data from OpenAI API or elsewhere
json_data = [
    {
        "report_id": "1017",
        "machine_id": "NMR-500",
        "product1_id": "Jojoba Oil",
        "product2_id": "Beeswax",
        "product3_id": "Glycerin",
        "result": 15.2,
        "unit": "ppm"
    },
    {
        "report_id": "1017",
        "machine_id": "FTIR-8400",
        "product1_id": "Almond Oil",
        "product2_id": "Vitamin E",
        "product3_id": None,
        "result": 3450.0,
        "unit": "1/cm"
    }
]

# Map the data using the OpenAI-assisted machine name matching
mapped_data = map_ids_using_openai(json_data)

# Generate the DataFrame
df_mapped = pd.DataFrame(mapped_data)

# Output the mapped data
print("Mapped Data Output:")
print(df_mapped.to_string(index=False))

# Save to CSV
output_csv_file = '/final_mapped_output_openai.csv'
df_mapped.to_csv(output_csv_file, index=False)

print(f"Final mapped data saved to {output_csv_file}")


Mapped Data Output:
report_id  machine_id  product1_id  product2_id  product3_id  result  unit
     1017          12            0            1          5.0    15.2     2
     1017          11            7            4          NaN  3450.0     3
Final mapped data saved to /final_mapped_output_openai.csv


In [72]:
import pickle as pkl
with open('/response_list1.pkl','rb') as f:
  d = pickle.load(f)

In [75]:
print(d[0])

```csv
report_id,machine_id,product1_id,product2_id,product3_id,result,unit
1017,NMR-500,Jojoba Oil,Beeswax,Glycerin,15.2,ppm
1017,FTIR-8400,Almond Oil,Vitamin E,,3450.0,1/cm
1017,PH-700,Almond Oil,Cetyl Alcohol,Vitamin E,7.5,pH
1017,GC-2010,Coconut Oil,Cetyl Alcohol,Glycerin,450.3,ppm
1017,XRD-6000,Coconut Oil,Beeswax,Glycerin,120.5,C
1017,TC-5000,Jojoba Oil,Beeswax,Glycerin,60.0,C
1017,MRX,Almond Oil,Vitamin E,,2.5,OD
1017,Alpha-300,Coconut Oil,Cetyl Alcohol,Glycerin,650.0,nm
1017,IC-2100,Jojoba Oil,Beeswax,Glycerin,50.5,mM
1017,VS-300,Jojoba Oil,Beeswax,Glycerin,3044.27,cP
1017,VS-300,Jojoba Oil,Gum,Glycerin,1814.4,cP
```


In [83]:
import pickle

# Replace with the path to your file
file_path = '/response_list1.pkl'

# Load the .pkl file and display its contents
try:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        print("Data type:", type(data))  # Print the type of the loaded data
        print("Content:", data)          # Print the content (if it’s not too large)
except Exception as e:
    print(f"Error loading the file: {e}")

import pandas as pd

if isinstance(data, pd.DataFrame):
    print(data.head())  # For pandas DataFrame
elif isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)  # Convert list of dicts to DataFrame
    print(df.head())
else:
    print(data[:5])  # Print first 5 items if it's a list or similar structure


Data type: <class 'list'>
Content: ['```csv\nreport_id,machine_id,product1_id,product2_id,product3_id,result,unit\n1017,NMR-500,Jojoba Oil,Beeswax,Glycerin,15.2,ppm\n1017,FTIR-8400,Almond Oil,Vitamin E,,3450.0,1/cm\n1017,PH-700,Almond Oil,Cetyl Alcohol,Vitamin E,7.5,pH\n1017,GC-2010,Coconut Oil,Cetyl Alcohol,Glycerin,450.3,ppm\n1017,XRD-6000,Coconut Oil,Beeswax,Glycerin,120.5,C\n1017,TC-5000,Jojoba Oil,Beeswax,Glycerin,60.0,C\n1017,MRX,Almond Oil,Vitamin E,,2.5,OD\n1017,Alpha-300,Coconut Oil,Cetyl Alcohol,Glycerin,650.0,nm\n1017,IC-2100,Jojoba Oil,Beeswax,Glycerin,50.5,mM\n1017,VS-300,Jojoba Oil,Beeswax,Glycerin,3044.27,cP\n1017,VS-300,Jojoba Oil,Gum,Glycerin,1814.4,cP\n```', '```csv\nreport_id,machine_id,product1_id,product2_id,product3_id,result,unit\n1025,GC-2010,Coconut Oil,Gum,Glycerin,750,ppm\n1025,LC-400,Jojoba Oil,Beeswax,Vitamin E,300,ug/mL\n1025,HPLC-9000,Almond Oil,Beeswax,Vitamin E,650,mg/L\n1025,MRX,Jojoba Oil,Gum,,1.5,OD\n1025,CM-215,Coconut Oil,, ,1500,uS/cm\n1025,T-905,

In [84]:
import os
import re
import pandas as pd
import docx2txt
import pdfplumber
import json
import pickle
from openai import AzureOpenAI

# Azure OpenAI credentials
gpt4o_endpoint = "https://fuchsopenai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2023-03-15-preview"
gpt4o_key = "377635a0dfdd4f01a1352ea785ea4537"

# Azure OpenAI model and client configuration
model_name = "gpt-4o-2024-08-06"
client = AzureOpenAI(
    azure_endpoint=gpt4o_endpoint,
    api_key=gpt4o_key,
    api_version="2024-02-01"
)

# Load the mapping files
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

# Load the .pkl file
def load_pkl_file(file_path):
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
            return data
    except Exception as e:
        print(f"Error reading the .pkl file: {e}")
        return None

# Function to send a request to OpenAI to find the closest match
def get_closest_machine_name_openai(machine_name, available_machines):
    """
    Function to get the closest machine name using OpenAI's language model.

    Parameters:
    - machine_name (str): The machine name from the input data.
    - available_machines (list): List of machine names from the mapping file.

    Returns:
    - str: Closest matching machine name as suggested by OpenAI.
    """
    # Build the prompt
    prompt = f"""
    I have a list of available machine names: {', '.join(available_machines)}.
    Based on this list, find the closest matching machine name for: "{machine_name}".
    Only provide the closest match.
    """

    # Call the OpenAI API
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}]
        )
        response_text = response.choices[0].message.content.strip()
        return response_text
    except Exception as e:
        print(f"Error generating text from OpenAI: {e}")
        return None

def map_ids_using_openai(response_data):
    mapped_data = []

    # Available machine names from the machine mapping file
    available_machine_names = machine_mapping['machine_name'].tolist()

    for record in response_data:
        mapped_record = record.copy()

        # Map machine_id using OpenAI
        machine_name = record['machine_id']
        if pd.notna(machine_name):
            machine_name_cleaned = machine_name.strip().lower()

            # Get the closest machine name using OpenAI
            closest_machine_name = get_closest_machine_name_openai(machine_name_cleaned, available_machine_names)

            if closest_machine_name:
                # Fetch the corresponding machine_id for the closest match
                machine_id_row = machine_mapping.loc[
                    machine_mapping['machine_name'].str.lower().str.strip() == closest_machine_name.lower()
                ]
                if not machine_id_row.empty:
                    mapped_record['machine_id'] = machine_id_row['machine_id'].values[0]
                else:
                    mapped_record['machine_id'] = None
                    print(f"Warning: Could not map machine name '{machine_name}' to machine_id.")
            else:
                mapped_record['machine_id'] = None
                print(f"Warning: Could not find a close match for machine name '{machine_name}' using OpenAI")
        else:
            mapped_record['machine_id'] = None

        # Map product1_id, product2_id, product3_id
        for i in range(1, 4):
            product_name = record.get(f'product{i}_id')
            if product_name:
                product_name_cleaned = product_name.strip().lower()
                product_id_row = product_mapping.loc[
                    product_mapping['product_name'].str.lower().str.strip() == product_name_cleaned
                ]
                if not product_id_row.empty:
                    mapped_record[f'product{i}_id'] = product_id_row['product_id'].values[0]
                else:
                    mapped_record[f'product{i}_id'] = None
                    print(f"Warning: Could not map product name '{product_name}' to product_id.")
            else:
                mapped_record[f'product{i}_id'] = None

        # Map unit
        unit_name = record['unit']
        if unit_name:
            unit_name_cleaned = unit_name.strip().lower()
            unit_id_row = unit_mapping.loc[
                unit_mapping['unit'].str.lower().str.strip() == unit_name_cleaned
            ]
            if not unit_id_row.empty:
                mapped_record['unit'] = unit_id_row['unit_id'].values[0]
            else:
                mapped_record['unit'] = None  # If not found
                print(f"Warning: Could not map unit name '{unit_name}' to unit_id.")
        else:
            mapped_record['unit'] = None

        mapped_data.append(mapped_record)

    return mapped_data

# Load the .pkl file data (replace the file path with the actual path)
file_path = '/response_list1.pkl'
pkl_data = load_pkl_file(file_path)

# Convert the list of CSV-like strings into structured data (assumes the data is CSV-like)
response_data = []
for csv_data in pkl_data:
    try:
        # Remove any text decorations (like ```csv) and read the CSV content
        csv_cleaned = re.sub(r'```csv|```plaintext|\n```', '', csv_data)
        response_data.append(pd.read_csv(pd.compat.StringIO(csv_cleaned)).to_dict(orient='records'))
    except Exception as e:
        print(f"Error parsing CSV data from the .pkl file: {e}")

# Flatten the list of lists into a single list of records
response_data = [record for sublist in response_data for record in sublist]

# Map the data using the OpenAI-assisted machine name matching
mapped_data = map_ids_using_openai(response_data)

# Generate the DataFrame
df_mapped = pd.DataFrame(mapped_data)

# Output the mapped data
print("Mapped Data Output:")
print(df_mapped.to_string(index=False))

# Save to CSV
output_csv_file = '/final_mapped_output_openai.csv'
df_mapped.to_csv(output_csv_file, index=False)

print(f"Final mapped data saved to {output_csv_file}")


Error parsing CSV data from the .pkl file: module 'pandas.compat' has no attribute 'StringIO'
Error parsing CSV data from the .pkl file: module 'pandas.compat' has no attribute 'StringIO'
Error parsing CSV data from the .pkl file: module 'pandas.compat' has no attribute 'StringIO'
Error parsing CSV data from the .pkl file: module 'pandas.compat' has no attribute 'StringIO'
Error parsing CSV data from the .pkl file: module 'pandas.compat' has no attribute 'StringIO'
Mapped Data Output:
Empty DataFrame
Columns: []
Index: []
Final mapped data saved to /final_mapped_output_openai.csv


In [92]:
data[2], len(data)

('```csv\nreport_id,machine_id,product1_id,product2_id,product3_id,result,unit\n1026,UV-2600,Almond Oil,,2.1,Abs\n1026,MRX,Almond Oil,Gum,Glycerin,3.8,OD\n1026,Alpha-300,Jojoba Oil,Cetyl Alcohol,,210.0,nm\n1026,FTIR-8400,Jojoba Oil,Beeswax,,1500.0,1/cm\n1026,GC-2010,Jojoba Oil,Cetyl Alcohol,,850,ppm\n1026,NMR-500,Jojoba Oil,Beeswax,,15,ppm\n1026,IC-2100,Coconut Oil,Vitamin E,,10,mM\n1026,MS-20,Almond Oil,Beeswax,,1500.0,m/z\n1026,XRD-6000,Coconut Oil,Vitamin E,,90.0,C\n1026,VS-300 (Trial 1),Almond Oil,Glycerin,,7359.54,cP\n1026,VS-300 (Trial 2),Almond Oil,Gum,Glycerin,7513.0,cP\n```',
 5)

In [124]:
import pickle
with open('/response_list_50.pkl','rb') as f:
  data = pickle.load(f)

'```csv\nreport_id,machine_id,product1_id,product2_id,product3_id,result,unit\n1077,IC-2100,Coconut Oil,Cetyl Alcohol,Glycerin,50.678,mM\n1077,FB-1000,Almond Oil,Beeswax,Vitamin E,0.650,mm\n1077,R-4500,Almond Oil,Beeswax,Glycerin,250.4,Pa-s\n1077,XRD-6000,Coconut Oil,Gum,Glycerin,30,°C Peak\n1077,MRX,Coconut Oil,Gum,,2.8,OD\n1077,CM-215,Almond Oil,Beeswax,Vitamin E,1500,uS/cm\n1077,PCR-96,Coconut Oil,Gum,Glycerin,25,Ct\n1077,Alpha-300,Almond Oil,Beeswax,Glycerin,450,nm\n1077,UV-2600,Coconut Oil,Cetyl Alcohol,Glycerin,1.8,Abs\n1077,VS-300,Coconut Oil,Vitamin E,,5027.61,cP\n1077,VS-300,Coconut Oil,Gum,Vitamin E,5292.83,cP\n```'

In [136]:
import csv

for i in range(len(data)):
  if i>200 and i<250:
    try:
      lines = data[i].strip().split('\n')
      lines = lines[1:]

      # Open a CSV file to write
      with open(f"/var/output9/output_{i}.csv", "w", newline='', encoding='utf-8') as csvfile:
          csv_writer = csv.writer(csvfile)

        # Iterate through each line and write it as a row in the CSV file
          for line in lines:
              csv_writer.writerow(line.split(','))
    except: continue

In [126]:
import os
import pandas as pd
import json
from openai import AzureOpenAI

# Azure OpenAI credentials
gpt4o_endpoint = "https://fuchsopenai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2023-03-15-preview"
gpt4o_key = "377635a0dfdd4f01a1352ea785ea4537"

# Azure OpenAI model and client configuration
model_name = "gpt-4o-2024-08-06"
client = AzureOpenAI(
    azure_endpoint=gpt4o_endpoint,
    api_key=gpt4o_key,
    api_version="2024-02-01"
)

# Load the mapping files
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

# Function to send a request to OpenAI to find the closest match
def get_closest_machine_name_openai(machine_name, available_machines):
    prompt = f"""
    I have a list of available machine names: {', '.join(available_machines)}.
    Based on this list, find the closest matching machine name for: "{machine_name}".
    Only provide the closest match.
    """

    # Call the OpenAI API
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}]
        )
        response_text = response.choices[0].message.content.strip()
        return response_text
    except Exception as e:
        print(f"Error generating text from OpenAI: {e}")
        return None

# Function to map machine, product, and unit IDs using OpenAI for machine name matching
def map_ids_using_openai(response_data):
    mapped_data = []

    # Available machine names from the machine mapping file
    available_machine_names = machine_mapping['machine_name'].tolist()

    for record in response_data:
        mapped_record = record.copy()

        # Map machine_id using OpenAI
        machine_name = record['machine_id']
        if pd.notna(machine_name):
            machine_name_cleaned = str(machine_name).strip().lower()

            # Get the closest machine name using OpenAI
            closest_machine_name = get_closest_machine_name_openai(machine_name_cleaned, available_machine_names)

            if closest_machine_name:
                # Fetch the corresponding machine_id for the closest match
                machine_id_row = machine_mapping.loc[
                    machine_mapping['machine_name'].str.lower().str.strip() == closest_machine_name.lower()
                ]
                if not machine_id_row.empty:
                    mapped_record['machine_id'] = machine_id_row['machine_id'].values[0]
                else:
                    mapped_record['machine_id'] = None
                    print(f"Warning: Could not map machine name '{machine_name}' to machine_id.")
            else:
                mapped_record['machine_id'] = None
                print(f"Warning: Could not find a close match for machine name '{machine_name}' using OpenAI")
        else:
            mapped_record['machine_id'] = None

        # Map product1_id, product2_id, product3_id
        for i in range(1, 4):
            product_name = record.get(f'product{i}_id')
            if pd.notna(product_name):
                product_name_cleaned = str(product_name).strip().lower()
                product_id_row = product_mapping.loc[
                    product_mapping['product_name'].str.lower().str.strip() == product_name_cleaned
                ]
                if not product_id_row.empty:
                    mapped_record[f'product{i}_id'] = product_id_row['product_id'].values[0]
                else:
                    mapped_record[f'product{i}_id'] = None
                    print(f"Warning: Could not map product name '{product_name}' to product_id.")
            else:
                mapped_record[f'product{i}_id'] = None

        # Map unit
        unit_name = record['unit']
        if pd.notna(unit_name):
            unit_name_cleaned = str(unit_name).strip().lower()
            unit_id_row = unit_mapping.loc[
                unit_mapping['unit'].str.lower().str.strip() == unit_name_cleaned
            ]
            if not unit_id_row.empty:
                mapped_record['unit'] = unit_id_row['unit_id'].values[0]
            else:
                mapped_record['unit'] = None  # If not found
                print(f"Warning: Could not map unit name '{unit_name}' to unit_id.")
        else:
            mapped_record['unit'] = None

        mapped_data.append(mapped_record)

    return mapped_data

# Function to process CSV files in a folder
def process_csv_folder(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    all_data = []

    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        print(f"Processing file: {file_path}")

        # Load the CSV file
        df = pd.read_csv(file_path)

        # Convert DataFrame rows to a list of dictionaries
        data = df.to_dict(orient='records')

        # Map the data using the OpenAI-assisted machine name matching
        mapped_data = map_ids_using_openai(data)

        # Append the mapped data to all_data list
        all_data.extend(mapped_data)

    return all_data

# Specify the folder containing the CSV files
csv_folder_path = '/var/output6'

# Process the CSV files and get the mapped data
mapped_data = process_csv_folder(csv_folder_path)

# Generate the final DataFrame
df_mapped = pd.DataFrame(mapped_data)

# Output the mapped data
print("Mapped Data Output:")
print(df_mapped.to_string(index=False))

# Save to a final CSV
output_csv_file = '/final_mapped_output_openai.csv'
df_mapped.to_csv(output_csv_file, index=False)

print(f"Final mapped data saved to {output_csv_file}")


Mapped Data Output:
Empty DataFrame
Columns: []
Index: []
Final mapped data saved to /final_mapped_output_openai.csv


In [127]:
import os
import pandas as pd
import time
from openai import AzureOpenAI

# Azure OpenAI credentials
gpt4o_endpoint = "https://fuchsopenai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2023-03-15-preview"
gpt4o_key = "377635a0dfdd4f01a1352ea785ea4537"

# Azure OpenAI model and client configuration
model_name = "gpt-4o-2024-08-06"
client = AzureOpenAI(
    azure_endpoint=gpt4o_endpoint,
    api_key=gpt4o_key,
    api_version="2024-02-01"
)

# Load the mapping files
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

# Retry OpenAI request in case of rate limit
def get_closest_machine_name_openai(machine_name, available_machines, retries=3, delay=5):
    """
    Function to get the closest machine name using OpenAI's language model.
    Retries in case of rate limiting.
    """
    prompt = f"""
    I have a list of available machine names: {', '.join(available_machines)}.
    Based on this list, find the closest matching machine name for: "{machine_name}".
    Only provide the closest match.
    """

    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}]
            )
            response_text = response.choices[0].message.content.strip()
            return response_text
        except Exception as e:
            if "429" in str(e):
                print(f"Rate limit hit. Retrying after {delay} seconds... (Attempt {attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"Error generating text from OpenAI: {e}")
                return None
    return None

# Function to map machine, product, and unit IDs using OpenAI for machine name matching
def map_ids_using_openai(response_data):
    mapped_data = []

    # Available machine names from the machine mapping file
    available_machine_names = machine_mapping['machine_name'].tolist()

    for record in response_data:
        mapped_record = record.copy()

        # Map machine_id using OpenAI
        machine_name = record['machine_id']
        if pd.notna(machine_name):
            machine_name_cleaned = str(machine_name).strip().lower()

            # Get the closest machine name using OpenAI with retry logic
            closest_machine_name = get_closest_machine_name_openai(machine_name_cleaned, available_machine_names)

            if closest_machine_name:
                # Fetch the corresponding machine_id for the closest match
                machine_id_row = machine_mapping.loc[
                    machine_mapping['machine_name'].str.lower().str.strip() == closest_machine_name.lower()
                ]
                if not machine_id_row.empty:
                    mapped_record['machine_id'] = machine_id_row['machine_id'].values[0]
                else:
                    mapped_record['machine_id'] = None
                    print(f"Warning: Could not map machine name '{machine_name}' to machine_id.")
            else:
                mapped_record['machine_id'] = None
                print(f"Warning: Could not find a close match for machine name '{machine_name}' using OpenAI")
        else:
            mapped_record['machine_id'] = None

        # Map product1_id, product2_id, product3_id
        for i in range(1, 4):
            product_name = record.get(f'product{i}_id')
            if pd.notna(product_name):
                product_name_cleaned = str(product_name).strip().lower()
                product_id_row = product_mapping.loc[
                    product_mapping['product_name'].str.lower().str.strip() == product_name_cleaned
                ]
                if not product_id_row.empty:
                    mapped_record[f'product{i}_id'] = product_id_row['product_id'].values[0]
                else:
                    mapped_record[f'product{i}_id'] = None
                    print(f"Warning: Could not map product name '{product_name}' to product_id.")
            else:
                mapped_record[f'product{i}_id'] = None

        # Map unit
        unit_name = record['unit']
        if pd.notna(unit_name):
            unit_name_cleaned = str(unit_name).strip().lower()
            unit_id_row = unit_mapping.loc[
                unit_mapping['unit'].str.lower().str.strip() == unit_name_cleaned
            ]
            if not unit_id_row.empty:
                mapped_record['unit'] = unit_id_row['unit_id'].values[0]
            else:
                mapped_record['unit'] = None  # If not found
                print(f"Warning: Could not map unit name '{unit_name}' to unit_id.")
        else:
            mapped_record['unit'] = None

        mapped_data.append(mapped_record)

    return mapped_data

# Function to process CSV files in a folder
def process_csv_folder(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    all_data = []

    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        print(f"Processing file: {file_path}")

        # Load the CSV file
        df = pd.read_csv(file_path)

        # Remove any invalid rows, such as empty or strange rows like '```'
        df = df.dropna(how="all")  # Drop rows where all elements are NaN
        df = df[~df.astype(str).apply(lambda x: x.str.contains('```')).any(axis=1)]  # Drop rows with '```'

        # Convert DataFrame rows to a list of dictionaries
        data = df.to_dict(orient='records')

        # Map the data using the OpenAI-assisted machine name matching
        mapped_data = map_ids_using_openai(data)

        # Append the mapped data to all_data list
        all_data.extend(mapped_data)

    return all_data

# Specify the folder containing the CSV files
csv_folder_path = '/var/output6'

# Process the CSV files and get the mapped data
mapped_data = process_csv_folder(csv_folder_path)

# Generate the final DataFrame
df_mapped = pd.DataFrame(mapped_data)

# Output the mapped data
print("Mapped Data Output:")
print(df_mapped.to_string(index=False))

# Save to a final CSV
output_csv_file = '/final_mapped_output_openai.csv'
df_mapped.to_csv(output_csv_file, index=False)

print(f"Final mapped data saved to {output_csv_file}")


Mapped Data Output:
Empty DataFrame
Columns: []
Index: []
Final mapped data saved to /final_mapped_output_openai.csv


In [137]:
import os
import pandas as pd
import time
from openai import AzureOpenAI

# Azure OpenAI credentials
gpt4o_endpoint = "https://fuchsopenai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2023-03-15-preview"
gpt4o_key = "377635a0dfdd4f01a1352ea785ea4537"

# Azure OpenAI model and client configuration
model_name = "gpt-4o-2024-08-06"
client = AzureOpenAI(
    azure_endpoint=gpt4o_endpoint,
    api_key=gpt4o_key,
    api_version="2024-02-01"
)

# Load the mapping files
machine_mapping = pd.read_csv("/machine_mapping.csv")
product_mapping = pd.read_csv("/product_mapping.csv")
unit_mapping = pd.read_csv("/unit_mapping.csv")

# Retry OpenAI request in case of rate limit
def get_closest_machine_name_openai(machine_name, available_machines, retries=3, delay=5):
    """
    Function to get the closest machine name using OpenAI's language model.
    Retries in case of rate limiting.
    """
    prompt = f"""
    I have a list of available machine names: {', '.join(available_machines)}.
    Based on this list, find the closest matching machine name for: "{machine_name}".
    Only provide the closest match.
    """

    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}]
            )
            response_text = response.choices[0].message.content.strip()
            return response_text
        except Exception as e:
            if "429" in str(e):
                print(f"Rate limit hit. Retrying after {delay} seconds... (Attempt {attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"Error generating text from OpenAI: {e}")
                return None
    return None

# Function to map machine, product, and unit IDs using OpenAI for machine name matching
def map_ids_using_openai(response_data):
    mapped_data = []

    # Available machine names from the machine mapping file
    available_machine_names = machine_mapping['machine_name'].tolist()

    for record in response_data:
        mapped_record = record.copy()

        # Map machine_id using OpenAI
        machine_name = record['machine_id']
        if pd.notna(machine_name):
            machine_name_cleaned = str(machine_name).strip().lower()

            # Get the closest machine name using OpenAI with retry logic
            closest_machine_name = get_closest_machine_name_openai(machine_name_cleaned, available_machine_names)

            if closest_machine_name:
                # Fetch the corresponding machine_id for the closest match
                machine_id_row = machine_mapping.loc[
                    machine_mapping['machine_name'].str.lower().str.strip() == closest_machine_name.lower()
                ]
                if not machine_id_row.empty:
                    mapped_record['machine_id'] = machine_id_row['machine_id'].values[0]
                else:
                    mapped_record['machine_id'] = None
                    print(f"Warning: Could not map machine name '{machine_name}' to machine_id.")
            else:
                mapped_record['machine_id'] = None
                print(f"Warning: Could not find a close match for machine name '{machine_name}' using OpenAI")
        else:
            mapped_record['machine_id'] = None

        # Map product1_id, product2_id, product3_id
        for i in range(1, 4):
            product_name = record.get(f'product{i}_id')
            if pd.notna(product_name):
                product_name_cleaned = str(product_name).strip().lower()
                product_id_row = product_mapping.loc[
                    product_mapping['product_name'].str.lower().str.strip() == product_name_cleaned
                ]
                if not product_id_row.empty:
                    mapped_record[f'product{i}_id'] = product_id_row['product_id'].values[0]
                else:
                    mapped_record[f'product{i}_id'] = None
                    print(f"Warning: Could not map product name '{product_name}' to product_id.")
            else:
                mapped_record[f'product{i}_id'] = None

        # Map unit
        unit_name = record['unit']
        if pd.notna(unit_name):
            unit_name_cleaned = str(unit_name).strip().lower()
            unit_id_row = unit_mapping.loc[
                unit_mapping['unit'].str.lower().str.strip() == unit_name_cleaned
            ]
            if not unit_id_row.empty:
                mapped_record['unit'] = unit_id_row['unit_id'].values[0]
            else:
                mapped_record['unit'] = None  # If not found
                print(f"Warning: Could not map unit name '{unit_name}' to unit_id.")
        else:
            mapped_record['unit'] = None

        mapped_data.append(mapped_record)

    return mapped_data

# Function to process CSV files in a folder
def process_csv_folder(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    all_data = []

    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        print(f"Processing file: {file_path}")

        # Load the CSV file
        try:
          df = pd.read_csv(file_path)

          # Remove any invalid rows, such as empty or strange rows like '```'
          df = df.dropna(how="all")  # Drop rows where all elements are NaN
          df = df[~df.astype(str).apply(lambda x: x.str.contains('```')).any(axis=1)]  # Drop rows with '```'

          # Convert DataFrame rows to a list of dictionaries
          data = df.to_dict(orient='records')

          # Map the data using the OpenAI-assisted machine name matching
          mapped_data = map_ids_using_openai(data)

          # Append the mapped data to all_data list
          all_data.extend(mapped_data)
        except: continue

    return all_data

# Specify the folder containing the CSV files
csv_folder_path = '/var/output6'

# Process the CSV files and get the mapped data
mapped_data = process_csv_folder(csv_folder_path)

# Generate the final DataFrame
df_mapped = pd.DataFrame(mapped_data)

# Rename the columns to the desired format
df_mapped.rename(columns={
    'product1_id': 'ing_1_id',
    'product2_id': 'ing_2_id',
    'product3_id': 'ing_3_id',
    'unit': 'unit_id'
}, inplace=True)

# Output the mapped data
print("Mapped Data Output:")
print(df_mapped.to_string(index=False))

# Save to a final CSV with the correct header
output_csv_file = '/final_mapped_output_openai.csv'
df_mapped.to_csv(output_csv_file, index=False)

print(f"Final mapped data saved to {output_csv_file}")

Processing file: /var/output6/output_88.csv
Processing file: /var/output6/output_132.csv
Processing file: /var/output6/output_122.csv
Processing file: /var/output6/output_57.csv
Processing file: /var/output6/output_133.csv
Processing file: /var/output6/output_64.csv
Processing file: /var/output6/output_69.csv
Processing file: /var/output6/output_108.csv
Processing file: /var/output6/output_124.csv
Processing file: /var/output6/output_87.csv
Processing file: /var/output6/output_97.csv
Processing file: /var/output6/output_110.csv
Processing file: /var/output6/output_123.csv
Processing file: /var/output6/output_54.csv
Processing file: /var/output6/output_144.csv
Processing file: /var/output6/output_131.csv
Processing file: /var/output6/output_139.csv
Processing file: /var/output6/output_52.csv
Processing file: /var/output6/output_66.csv
Processing file: /var/output6/output_147.csv
Processing file: /var/output6/output_149.csv
Processing file: /var/output6/output_141.csv
Processing file: /v